# Exploring ReacLib with [`pynucastro`](https://github.com/pynucastro/pynucastro)

In this pragmatic tutorial, we learn about both the JINA-CEE ReacLib database and `pynucastro` by seeing how the latter can be used to interface with the former.

In [ ]:
#Do the necessary importing/setup
#Your Python environment will need everything in pynucastro/requirements.txt for this to work.
%matplotlib inline 
import pynucastro as pyna
from pynucastro.rates import Library, RateFilter

### Loading a ReacLib Snapshot

In [ ]:
#Create a Library object representing the data in a reaclib snapshot
#'ReaclibV2.2_20161114' is the V2.2 snapshot file available here: http://reaclib.jinaweb.org/library.php?action=viewsnapshots
snapshot_file='ReaclibV2.2_20161114'
#This Library object instance `reaclib22` is a collection of rates, 
#calculated using the reaction rate fit parameters stored in a ReacLib snapshot
reaclib22 = Library(libfile=snapshot_file)

In [ ]:
#Let's look at the rates making up this library
rates = reaclib22.get_rates()
for i in range(len(rates)):
    #We can print string representations of each rate like this
    print(i, rates[i])

In [ ]:
#Look at how many rates!
print(len(rates))

### `Rate`s, `RateFilter`s, and ReacLib sets

To learn more, let's look at a particular Rate object for $\mathrm{O}^{15} (p,\gamma) \mathrm{F}^{16}$ in this Library

In [ ]:
#First, to find a rate `pynucastro` helpfully includes RateFilters.  
#These can be used to quickly find one rate out of 10s of thousands using human-friendly
#text searches on nuclides and other properties.  See docs for details
o15pgf16_rf   = RateFilter(reactants=['o15', 'p'], products=['f16'])
o15pgf16_lib = reaclib22.filter(o15pgf16_rf)
o15pgf16_rate = o15pgf16_lib.get_rates()[0]

#Found it! Let's explore.  First, you can print a string representation easily:
print(o15pgf16_rate)

#You can access various properties of the rate
print(o15pgf16_rate.original_source) #ReacLib snapshot source text
#Much of this can be broken out into pieces:
print("Chapter: {}".format(o15pgf16_rate.chapter)) #ReacLib Chapter
print("Q-value (MeV): {}".format(o15pgf16_rate.Q))
print("Reactants: {}".format(o15pgf16_rate.reactants)) #reactants
print("Products:  {}".format(o15pgf16_rate.products)) #products

Note that the $\mathrm{O}^{15} (p, \gamma) \mathrm{F}^{16}$ rate is calculated using one ReacLib set.  Each set in ReacLib provides 7 coefficients for their rate-fitting equation:

$$
\begin{equation}
    \lambda = \exp \left[ a_0 
        + \sum_{i=1}^5 a_i T_9^{\frac{2i-5}{3}}
        + a_6 \ln T_9 
    \right]
\end{equation}
$$

A rate may consist of several sets, which are summed together to get the total rate.  This is because the experimental inputs for rate calculations and the theoretical machinery from going from this experimental data to usable, temperature-dependent rates gets built up from different components.  The primary components, as discussed [here](http://reaclib.jinaweb.org/help.php?topic=rate_fitting&intCurrentNum=0) on the reaclib website, are (1) charge-induced, non-resonant, (2) neutral-induced, non-resonant, and (3) narrow resonances.  As you can see on the website, experimental measurements and nuclear properties contribute to the coefficients.  In addition, different types of sets may use different fitting forms based on theory. 

You can see the sets composing a rate with:

In [ ]:
print('List of rate sets: {}'.format(o15pgf16_rate.sets))
#It's only one set for this rate, let's look at it:
rl_set = o15pgf16_rate.sets[0]
print('The a_i coefficients: {}'.format(rl_set.a))
print('Is this a resonant set?: {}'.format(rl_set.resonant))
print('Is this a weak set?: {}'.format(rl_set.weak))
print('Is this a reverse-derived set?: {}'.format(rl_set.reverse))
print('Rate reaclib source label: {}'.format(rl_set.label)) #labels can be associated with sources

Source label: the source label in ReacLib can be associated with papers/sources that the rate fit was derived from.  To find these details, see the ReacLib [labels page](https://jinaweb.org/reaclib/db/labels.php).  For example, for this rate we have:  
`rpsm`: T. Rauscher, NON-SMOKER Hauser Feshbach rates with Audi Wapastra masses, private communication, 1999

Reverse flag: Sets with this flag are for "reverse rates in which detailed balance (without partition functions) was used to derive the reaction rate. Therefore, rates with this flag must be corrected to include partition function modifications." (from [here](http://reaclib.jinaweb.org/help.php?topic=reaclib_format&intCurrentNum=0))

`pynucastro` even lets you see the rate/fit in a plot:

In [ ]:
o15pgf16_rate.plot(Tmin=0.5e9)

[1]: http://reaclib.jinaweb.org/o15(p,g)f16/rpsm/
This compares well, as one would hope, with the [rate plot][1] on the ReacLib website.

As an example of a rate with multiple sets, consider $\mathrm{C}^{12} (\alpha, \gamma) \mathrm{O}^{16}$.

In [ ]:
c12ago16_rf  = RateFilter(reactants=['c12', 'a'], products=['o16'])
c12ago16_lib = reaclib22.filter(c12ago16_rf)
c12ago16_rate = c12ago16_lib.get_rates()[0]
print('rate string: {}'.format(c12ago16_rate))
print('Look, two sets:\n{}'.format(c12ago16_rate.sets))
print('    set 1 coeffs: {}'.format(c12ago16_rate.sets[0].a))
print('    set 1 full label: {}'.format(c12ago16_rate.sets[0].labelprops))
print('    set 2 coeffs: {}'.format(c12ago16_rate.sets[1].a))
print('    set 2 full label: {}'.format(c12ago16_rate.sets[1].labelprops))

I'm not actually sure what these two sets correspond to.  They must all be charge-induced (it's an (a,g) rxn) or narrow resonance, but they aren't in the proper form for either of these set types.  Neither set is flagged as resonant (`r`), but they have non-zero $a_1$ fit parameters, which is only true of resonant sets (again, see the [website docs](http://reaclib.jinaweb.org/help.php?topic=rate_fitting&intCurrentNum=0))

### Reverse-derived Rates and Partition Functions

We've seen reverse-derived rates and discussion of partition functions above.  These are key concepts in the astrophysical concept, so we use this section to explore them in more detail.

The ReacLib database draws from many sources, but part of its power and utility is that it imposes the same formalism on these sources.  That formalism, and indeed much of the ReacLib concept to my knowledge, is largely drawn from [Rausher & Thielemann 2000](https://arxiv.org/abs/astro-ph/0004059), a landmark publication in the field of nuclear astrophysics.  It focuses on the needs of astrophysical modelers, which often differ from those of nuclear theorists/modelers.  This formalism dates back, in part, to giants in the field like Fowler (such as [this](http://adsabs.harvard.edu/abs/1967ARA%26A...5..525F) from 1967).  Part of the formalism includes calculating and fitting rates in the form seen before:

$$
\begin{equation}
    \lambda = \exp \left[ a_0 
        + \sum_{i=1}^5 a_i T_9^{\frac{2i-5}{3}}
        + a_6 \ln T_9 
    \right]
\end{equation}
$$

This is the "rate".  For convenience, it is calculated as this $\lambda$ coefficient.  This is a reduced rate of sorts, and can have different units depending on the reaction type.  In a network of nuclear reaction ODEs, you'll have equations like the following for reactions of the form A+B-->C+D (this is from eqn. 2 in the 2010 reaclib paper):

$$
\begin{equation}
    -\frac{1}{N_A}\frac{dY_A}{dt} = -\frac{1}{N_B}\frac{dY_B}{dt}
    = \frac{1}{N_C}\frac{dY_C}{dt} = \frac{1}{N_D}\frac{dY_D}{dt} \\
    = \frac{Y_A^{N_A}}{N_A!}\frac{Y_B^{N_B}}{N_B!} \rho_\mathrm{baryon}^\nu \lambda
\end{equation}
$$

Here, $Y_I$ are molar abundances per gram of nuclide $I$, $N_I$ is the number of nuclides of $I$ produced or destroyed in the reaction, and $\nu=N_A + N_B - 1$.  Part of the convenience of $\lambda$ is that it can represent rates for many different types of reactions, which may have different units.  So for unary rates (e.g. a decay), $\lambda = 1/\tau$ is the inverse of the decay timescale with units of $\mathrm{s}^{-1}$.  We can see in this case, $\nu=1+0-1=0$, so the density proportionality disappears, as expected for a unary rate.  For binary rates, $\lambda=N_\mathrm{Avogadro} \langle \sigma v \rangle$ has units of $\mathrm{cm}^3 \mathrm{s}^{-1} \mathrm{mol}^{-1}$, and trinary rates have $\mathrm{cm}^6 \mathrm{s}^{-1} \mathrm{mol}^{-2}$ (and $\nu=N_1+N_2+N_3-1=3-1=2$, so we obtain the expected $\rho^2$ dependence).

So that's a bit of context on $\lambda$ and how it relates to calculting nuclear reaction rates.  At this point, astrophysicist diverge a bit from our purely nuclear colleagues.  We tend to be concerned with a "target nucleus", literally a target for their cool beams of things like protons to smash into.  This nucleus is just chilling, i.e. it's in the ground state.  Astrophysical nuclei don't usually get to chill, expecially if they're nuclei of interest to modelers.  Such nuclei tend to be participating in explosions with a background temperature that is not $T=0$.  One consequence is that it's possible a reaction is not proceeding on a ground-state target but on a target with potentially populated excited states.  [Fowler, Caughlan, and Zimmermann 1967](http://adsabs.harvard.edu/abs/1967ARA%26A...5..525F) provide a nice discussion of this and how it leads to the derivation of temperature-dependent partition functions

$$
    G(T) = \sum_{I^*} g_{I^*} \exp(-E_{I^*}/k_B T),
$$

where we sum over all the _relevant_ excited states $I^*$ of nuclide $I$.  A state is relevant if it lives longer than the reaction timescale.  See FCZ67 for details.  The point for us is that partition functions have a crucial utility in astrophysics: properly deriving inverse rates from known rates.

If you want to derive an inverse rate $\lambda_{C+D \rightarrow A+B}$ from a known rate $\lambda_{A+B \rightarrow C+D}$, it is common to do so by assuming "detailed balance," which means you're assuming the reaction with the known rate is in equilibrium with the reverse rate, thus the "forward" (known) rate _is_ simply the reverse rate with a flipped sign.

You will be very unhappy if you do this in an astrophysical context and don't take into account potentially populated excited states participating in the reaction equilibrium.  If you look into the original sources/derivations, you'll find this comes out as multiplying by the ratio of partition functions:

$$
    \lambda_{C+D \rightarrow A+B} \sim \frac{G_{A}}{G_{D}} \lambda_{A+B \rightarrow C+D}
$$

Technically, I believe $G_{A}$ should be something like $G_{A} G_{B}$ and similarly $G_D$ should be $G_C G_D$.  But in practice reactions are of the form A(b,c)D, where b and c are lighter things like protons, neutrons, helium, and even massless things like photons.  It makes some sense that these don't tend to have excited states participating in the equilibrium and can have their parition functions taken as 1.  The main point here is that to properly derive a rate from a known inverse rate, astrophysicists must account for these excited states as encoded by the temperature-dependent partition functions.  Rauscher and Thielemann 2000 provided partition functions for many isotopes, normalized to their ground state spin (so $g=1$ if there are no excited state contributions at the given T).  Due to this normalization, it's common for the rate calculation to multiply by ground state spin in the form $2j_{\mathrm{gs},X}+1$, something like:

$$
    \lambda_{C+D \rightarrow A+B} 
    \sim \frac{(2j_{\mathrm{gs},A}+1)g_{A}}{(2j_{\mathrm{gs},D}+1)g_{D}} \lambda_{A+B \rightarrow C+D},
$$
where $g_{X}(T)$ are the ground-state-normalized partition functions.  So **your model calculations may need to know about ground-state spins**.  It's tricky.  Sometimes partition functions are provided with the spin information encoded, other times not.  Just be wary of this.  If $G=1$ at low $T$, then you probably need to multiply by $(2J_\mathrm{gs}+1)$.

We now have enough background to understand how to treat the "reverse" flag in the ReacLib database.  We'll see this in practice in the next section.

### Understanding a ReacLib Reaction Rate Calculation with `pynucastro`

With `pynucastro`, we can break a rate calculation into its component parts to better understand the calculation and data that are needed for it.

TODO: Write this up after you implement partition functions in `pynucastro`.